In [46]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
import pandas as pd 
import numpy as np


sys.path.append(str(Path("../..").resolve()))

from src.constants import model_data_dir, raw_data_dir, raw_data_name
from src.utils import add_lagged_targets, add_seasonal_features

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
factors = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=1)
factors.set_index("Date", inplace=True)
categories = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=2)
outcomes = pd.read_excel(raw_data_dir / raw_data_name, sheet_name=3)

In [88]:
sc_means = (factors.reset_index()
     .melt("Date", var_name = "Variable")
     .merge(categories, on="Variable")
     .groupby(["Date", "Subcategory"])
     .mean()
     .reset_index()
)
    

factors.reset_index().melt("Date", var_name = "Variable").merge(sc_means)

,Date,Variable,value,Subcategory
0,2000-12-26,V2X,NaN,Inflation
1,2000-12-26,V2X,NaN,Policy Uncertainty
2,2000-12-26,V2X,NaN,Sentiment
3,2000-12-26,V2X,NaN,VRP
4,2000-12-26,MSCI EM Vol,NaN,Inflation
...,...,...,...,...
418727,2006-11-22,EURUSD ATM Vol 1Y-3M,7.150,TERM/CARRY
418728,2010-09-02,USDJPY ATM Vol 1Y-3M,13.340,TERM/CARRY
418729,2010-09-02,EURUSD ATM Vol 1Y-3M,13.340,TERM/CARRY
418730,2020-07-02,USDJPY ATM Vol 1Y-3M,7.215,TERM/CARRY


In [77]:
factors.iloc[1:factors.shape[0]-1]

,VIX,V2X,MSCI EM Vol,MOVE,Oil Vol,Gold Vol,HY Vol,JPM Global FX Vol,JPM G7 FX Vol,JPM EM FX Vol,...,EURUSD ATM Vol 1Y-3M,G10 Carry Trade Index,JPY/USD Carry,GBP/USD Carry,EUR/USD Carry,CAD/USD Carry,AUD/USD Carry,NZD/USD Carry,CHF/USD Carry,Copper 1 year carry
Date,,,,,,,,,,,,,,,,,,,,,
2000-05-31,23.65,25.2600,NaN,103.10,NaN,NaN,NaN,12.11,12.56,9.54,...,13.275,110.382,97.7475,90.5178,76.8011,101.7900,93.2954,85.8620,77.4453,NaN
2000-06-01,22.36,24.1400,NaN,101.26,NaN,NaN,NaN,11.95,12.40,9.38,...,13.175,110.611,96.9475,90.0934,76.2372,102.1823,93.2278,85.7492,77.1084,NaN
2000-06-02,21.48,22.7600,NaN,98.23,NaN,NaN,NaN,11.96,12.42,9.20,...,13.400,110.789,97.3623,91.0199,77.4666,103.1199,94.4805,87.2503,78.2057,NaN
2000-06-05,22.71,23.4500,NaN,96.08,NaN,NaN,NaN,11.86,12.33,9.05,...,13.175,110.838,97.9722,91.6087,77.5905,102.9840,94.8368,87.8504,78.2794,NaN
2000-06-06,23.05,24.1600,NaN,99.32,NaN,NaN,NaN,12.10,12.61,9.14,...,13.250,110.328,99.3638,92.1372,78.1483,103.0993,95.9269,88.4134,79.0201,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-23,16.32,18.5490,39.15,55.60,NaN,13.4200,4.90,6.81,6.09,8.86,...,6.050,221.771,65.4068,93.2190,89.4100,128.7365,189.2267,217.7832,105.8435,3.55
2021-06-24,15.97,17.3518,17.65,54.83,NaN,13.0375,5.28,6.62,5.93,8.58,...,6.015,221.912,65.4594,92.9385,89.4526,128.5486,189.4258,218.1859,105.9092,3.45
2021-06-25,15.62,16.6823,17.37,55.58,NaN,12.6175,5.05,6.45,5.74,8.48,...,5.960,221.949,65.5295,92.6514,89.4726,128.8834,189.5998,218.6194,105.9633,4.00


In [99]:
sc_medians = factors.reset_index().melt("Date", var_name = "Variable", value_name="sc_value").merge(categories, on="Variable").groupby(["Date", "Subcategory"]).agg("mean").reset_index()


dd = factors.reset_index().melt("Date", var_name = "Variable").merge(sc_medians)    

In [37]:
lag = 7
lagged_targets = pd.DataFrame({f"lag_{i}": outcomes["value_1d_fwd_rel_d"].shift(i)[lag:] for i in range(1, lag+1)})
lagged_targets

,lag_1,lag_2,lag_3,lag_4,lag_5,lag_6,lag_7
7,0.0,0.0,0.0,1.0,0.0,1.0,0.0
8,1.0,0.0,0.0,0.0,1.0,0.0,1.0
9,1.0,1.0,0.0,0.0,0.0,1.0,0.0
10,0.0,1.0,1.0,0.0,0.0,0.0,1.0
11,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
5300,1.0,0.0,1.0,0.0,0.0,0.0,1.0
5301,1.0,1.0,0.0,1.0,0.0,0.0,0.0
5302,1.0,1.0,1.0,0.0,1.0,0.0,0.0
5303,0.0,1.0,1.0,1.0,0.0,1.0,0.0


In [73]:
import random
factors.pct_change().apply(random.choice, axis = 1)

Date
2000-05-30         NaN
2000-05-31         NaN
2000-06-01    0.038952
2000-06-02    0.004137
2000-06-05    0.000000
                ...   
2021-06-24    0.000476
2021-06-25    0.000000
2021-06-28   -0.000609
2021-06-29    1.000000
2021-06-30    0.000131
Length: 5306, dtype: float64